# CMR - Single List Length, Subjectwise
Now we'll consider the consequences of computing fits subject by subject.

## Load Data

In [1]:
from instance_cmr.datasets import prepare_murddata
from instance_cmr.model_fitting import cmr_murd_likelihood
from instance_cmr.model_fitting import cmr_murd_objective_function
from instance_cmr.model_fitting import visualize_fit
from instance_cmr.models import CMR

murd_trials0, murd_events0, murd_length0 = prepare_murddata(
    '../../data/MurdData_clean.mat', 0)

subject_trial_count = 80 # Each subject gets 80 trials a piece.

murd_events0.head()

,subject,list,item,input,output,study,recall,repeat,intrusion
0,1,1,1,1,5.0,True,True,0,False
1,1,1,2,2,7.0,True,True,0,False
2,1,1,3,3,NaN,True,False,0,False
3,1,1,4,4,NaN,True,False,0,False
4,1,1,5,5,NaN,True,False,0,False


## Fitting

In [2]:
from scipy.optimize import differential_evolution
from numba.typed import List
import numpy as np

free_parameters = [
    'encoding_drift_rate',
    'start_drift_rate',
    'recall_drift_rate',
    'shared_support',
    'item_support',
    'learning_rate',
    'primacy_scale',
    'primacy_decay',
    'stop_probability_scale',
    'stop_probability_growth',
    'choice_sensitivity']

lb = np.finfo(float).eps
ub = 1-np.finfo(float).eps

bounds = [
    (lb, ub),
    (lb, ub),
    (lb, ub),
    (lb, ub),
    (lb, ub),
    (lb, ub),
    (lb, 100),
    (lb, 100),
    (lb, ub),
    (lb, 10),
    (lb, 10)
]

In [ ]:
results = []

for subject, start_index in enumerate(range(0, len(murd_trials0), subject_trial_count)):
    print(subject)

    # cost function to be minimized
    # ours scales inversely with the probability that the data could have been 
    # generated using the specified parameters and our model
    cost_function = cmr_murd_objective_function(
        List([murd_trials0[start_index:start_index+subject_trial_count]]), 
        {'item_counts': List([murd_length0])}, free_parameters)

    results.append(differential_evolution(cost_function, bounds, disp=False))

## Results
This time we're less interested in the fitted parameters than in the final function values. We'll convert them into AIC scores and plot the distribution. 

```
[1431.4267075699668, 1396.4568088600927, 1147.4295324553746, 1410.7998248955914, 1236.7779925507152, 1514.3310664471815, 1125.9791582947662, 1112.3336678272258, 1806.6054973034454, 1619.0006744729635, 1099.97072482067, 1216.7252617831628, 1469.324493886911, 1638.259094912972, 1059.4091208273853]
```

In [6]:
function_values = [result.fun for result in results]
print(function_values)

[1431.4267075699668, 1396.4568088600927, 1147.4295324553746, 1410.7998248955914, 1236.7779925507152, 1514.3310664471815, 1125.9791582947662, 1112.3336678272258, 1806.6054973034454, 1619.0006744729635, 1099.97072482067, 1216.7252617831628, 1469.324493886911, 1638.259094912972, 1059.4091208273853]
